In [6]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.graph.message import add_messages

from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from dotenv import load_dotenv
from dataclasses import dataclass
from typing import Annotated, List
from typing_extensions import TypedDict
import os
load_dotenv()
google_api_key=os.getenv('google_api_key')

True

In [7]:
google_api_key=os.getenv('google_api_key')

In [ ]:
GEMINI_MODEL='gemini-2.0-flash'
llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model=GEMINI_MODEL, temperature=0.3)

In [ ]:
class State(TypedDict):
    messages: Annotated[List, add_messages]

from composio_langgraph import Action, ComposioToolSet, App
composio_toolset = ComposioToolSet(api_key="ui9jbd366fmtu1y6ekotd")
tools = composio_toolset.get_tools(apps=[App.GOOGLECALENDAR])

In [ ]:
llm_with_tools = llm.bind_tools(tools)
graph_builder = StateGraph(State)
tool_node = ToolNode(tools=tools)


In [ ]:
def chatbot(state: State):
    """ travel assistant that answers user questions about their trip.
    Depending on the request, leverage which tools to use if necessary."""
    return {"messages": [llm_with_tools.invoke(state['messages'])]}

In [ ]:
graph_builder.add_node("chatbot", chatbot)

        
graph_builder.add_node("tools", tool_node)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.set_entry_point("chatbot")

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
memory=MemorySaver()
graph=graph_builder.compile(checkpointer=memory)

In [ ]:
def chat(input:str):
    config = {"configurable": {"thread_id": "1"}}
    response=graph.invoke({'messages':HumanMessage(content=str(input))},config)
    return response['messages'][-1].content

In [ ]:
chat('list events starting with today')

In [ ]:
from pydantic_ai.mcp import MCPServerStreamableHTTP
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage
from dataclasses import dataclass

@dataclass
class Message_state:
    messages:list[ModelMessage]

class Agent_MCP:
    def __init__(self, mcp_server_url:str, api_keys:dict):
        self.mcp_server_url=mcp_server_url
        self.api_keys=api_keys
        self.tools=ComposioToolSet(api_key=api_keys['composio_key'])
        self.tool_shemas={
            'Notion Manager':{tool.name:tool for tool in self.tools.get_action_schemas(apps=[App.NOTION])}}
        self.memory=Message_state(messages=[])
        self.llm=OpenAIModel('gpt-4.1-nano',provider=OpenAIProvider(api_key=api_keys['openai_api_key']))
        self.mcp_server=MCPServerStreamableHTTP(self.mcp_server_url)
        self.agent=Agent(self.llm, mcp_servers=[self.mcp_server])

    async def main(self,query:str):
        async with self.agent.run_mcp_servers():  
            result = self.agent.run_sync(query, message_history=self.memory.messages)
            self.memory.messages=result.all_messages()
        return result.output

    def reset(self):
        self.memory.messages=[]